In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import os 
import random
import math
import numpy as np
import pandas as pd
import pydicom

from skimage import io
from skimage import measure
from skimage import transform, exposure
from skimage.transform import resize

from tqdm import tqdm
from scipy import ndimage

import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, load_model, Model
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, UpSampling2D, Conv2DTranspose, concatenate, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras import optimizers, regularizers
from keras import applications

# Run time parameters

In [ ]:
os.listdir('../input')

In [ ]:
FRAC = 0.1                                                                 # FRACtion of input to be used for training
TRAIN_VAL_SPLIT = 0.9                                                      # Proportion of Training to Validation set
EPOCH = 5
SAMPLE_N = 2
BATCH_SIZE = 8

IMG_SIZE = 256
N_CHANNEL = 3
THRESHOLD = 0.5                                                           # probability threshold for prediction 
RANDOM_STATE = SAMPLE_N
DATA_INPUT_DIR = '/kaggle/input'                                          # path containing inputs

# Helper Functions

In [ ]:
# function to read annotn file and put it into a dataframe
def read_annotn(file_path=''):
    annotn = pd.read_csv(file_path)
    annotn = annotn.reset_index(drop=True)
    return annotn

# function to (1) sample input (2) split the input into train val 
def get_train_val_df(frac=FRAC, train_val_split=TRAIN_VAL_SPLIT, random_state=RANDOM_STATE, file_path=''):
    df_annotn = read_annotn(file_path=file_path)
    df_id_label = df_annotn.fillna(0)
    df_id_label = df_id_label.sample(frac=frac, random_state=RANDOM_STATE)
    train_sample = round(train_val_split*df_id_label.shape[0])
    df_train = df_id_label[:train_sample]
    df_val = df_id_label[train_sample:]
    return df_train,df_val

# loss function for training
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

# metrics function for training
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * dice_coef_loss(y_true, y_pred)

def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# function to put training scores into a dataframe
def update_df_score(df, history, model):
    for i in range(EPOCH): 
        df = df.append( { 'epoch' : i,
                         'model': model,
                         'train_loss' : history.history['loss'][i], 
                         'val_loss' : history.history['val_loss'][i], 
                         'train_accuracy' : history.history['acc'][i], 
                         'val_accuracy' : history.history['val_acc'][i],
                         'train_dice_coef' : history.history['dice_coef'][i], 
                         'val_dice_coef' : history.history['val_dice_coef'][i],
                         'train_mean_iou' : history.history['mean_iou'][i], 
                         'val_mean_iou' : history.history['val_mean_iou'][i]
                         } , ignore_index=True)
    return df

# function to convert mask to list of x,y,w,h
def convert_mask_to_xywh (mask, threshold=THRESHOLD):
    component = mask[:, :, 0] > THRESHOLD
    component = measure.label(component)
    xywh_list = []
    for region in measure.regionprops(component):
        y, x, y2, x2 = region.bbox
        height = y2 - y
        width = x2 - x
        xywh_list.append([x, y, width, height])
    return xywh_list

# Generator class

In [ ]:
# generator class for generating batches of (1) image  and mask array for training and validation (2) image array for prediction 
class datagen(keras.utils.Sequence):
    def __init__(self, df_X_y, im_size=IMG_SIZE, batch_size=32, predict=False, augment=True):
        self.im_size = im_size
        self.df_X_y = df_X_y
        self.batch_size = batch_size
        self.predict = predict
        self.augment = augment
        
    def __len__(self):
        return int(math.ceil(self.df_X_y.shape[0] / float(self.batch_size)))

    def __getitem__(self, idx):
        batch = self.df_X_y[idx * self.batch_size:(idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, IMG_SIZE, IMG_SIZE, N_CHANNEL))
        if self.predict:
            for i, row in enumerate(batch.iterrows()):
                patientId = row[1]['patientId']
                dcm_file = DATA_INPUT_DIR + '/stage_2_test_images/%s' %patientId
                dcm_data = pydicom.read_file(dcm_file)
                img =  dcm_data.pixel_array
                img = exposure.equalize_hist(img)
                img = resize(img, (IMG_SIZE, IMG_SIZE), mode='reflect')
                img = img - img.mean()
                img = img / img.std()
                img = np.expand_dims(img, -1) if N_CHANNEL == 1 else np.stack((img,)*3, axis=-1)            
                X[i,] = img
            return np.array(X), batch['patientId']
        else:
            masks = np.zeros((self.batch_size, IMG_SIZE, IMG_SIZE, N_CHANNEL))
            for i, row in enumerate(batch.iterrows()):
                patientId = row[1]['patientId']
                dcm_file = DATA_INPUT_DIR + '/stage_2_train_images/%s.dcm' %patientId 
                dcm_data = pydicom.read_file(dcm_file)
                img =  dcm_data.pixel_array
                img = exposure.equalize_hist(img)
                img = resize(img, (IMG_SIZE, IMG_SIZE), mode='reflect')
                img = img - img.mean()
                img = img / img.std()
                img = np.expand_dims(img, -1) if N_CHANNEL == 1 else np.stack((img,)*3, axis=-1)
                msk = np.zeros(dcm_data.pixel_array.shape)
                for key, (_, x, y, width, height, _) in self.df_X_y.loc[self.df_X_y['patientId'] == patientId].iterrows():
                    x, y, w, h = (int(x), int(y), int(width), int(height))
                    msk[y:y+h, x:x+w] = 1 
                msk = resize(msk, (IMG_SIZE, IMG_SIZE), mode='reflect') > THRESHOLD
                msk = np.expand_dims(msk, -1) if N_CHANNEL == 1 else np.stack((msk,)*3, axis=-1)
                if self.augment and random.random() > 0.5:
                    img = np.fliplr(img)
                    msk = np.fliplr(msk)
                    angle = random.random()*15
                    img = ndimage.rotate(img, angle, reshape=False, order=1)
                    msk = ndimage.rotate(msk, angle, reshape=False, order=1)
                masks[i,] = msk
                X[i,] = img
            return np.array(X), np.array(masks)

# Models : (1) UNET+VGG16 (2) UNET+INCEPTIONRESNET (3) UNET+DENSENET (4) UNET+RESNET

In [ ]:
# UNET with initialization from VGG16
def get_unet_vgg(LR=1e-4):
    from keras.applications.vgg16 import VGG16
    img_input = Input( (IMG_SIZE, IMG_SIZE, N_CHANNEL))
    vgg16_base = VGG16(input_tensor=img_input, include_top=False)
    conv1 = vgg16_base.get_layer(index=2).output
    conv2 = vgg16_base.get_layer(index=5).output
    conv3 = vgg16_base.get_layer(index=9).output
    pool3 = vgg16_base.get_layer(index=10).output

    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool3)
    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv4)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool4)
    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv5)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5)

    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool5)
    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv6)
    pool6 = MaxPooling2D((2, 2), strides=(2, 2))(conv6)

    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool6)
    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv7)
    
    up8 = concatenate([Conv2DTranspose(384, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same' )(conv7), conv6], axis=3)
    conv8 = Conv2D(384, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up8)

    up9 = concatenate([Conv2DTranspose(256, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv8), conv5], axis=3)
    conv9 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up9)

    up10 = concatenate([Conv2DTranspose(192, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same' )(conv9), conv4], axis=3)
    conv10 = Conv2D(192, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up10)

    up11 = concatenate([Conv2DTranspose(128, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv10), conv3], axis=3)
    conv11 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up11)

    up12 = concatenate([Conv2DTranspose(64, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv11), conv2], axis=3)
    conv12 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up12)

    up13 = concatenate([Conv2DTranspose(32, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv12), conv1], axis=3)
    conv13 = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up13)

    conv13 = Conv2D(N_CHANNEL, (1, 1))(conv13)
    conv13 = Activation("sigmoid")(conv13)
    model = Model(img_input, conv13)
    Adam = optimizers.Adam(lr=LR)
    model.compile(optimizer=Adam, loss=dice_coef_loss, metrics=['accuracy', dice_coef, mean_iou])
    model.name = 'unet+vgg'
    return model

In [ ]:
# UNET with initialization from INCEPTIONRESNET
def get_unet_IR(LR=1e-4):
    from keras.applications.inception_resnet_v2 import InceptionResNetV2
    img_input = Input( (IMG_SIZE, IMG_SIZE, N_CHANNEL))

    IR_model = InceptionResNetV2(weights='imagenet', input_tensor=img_input, include_top=False)
    conv1 = IR_model.get_layer(index=3).output 
    conv1 = keras.layers.UpSampling2D(2)(conv1)
    conv1 = keras.layers.Lambda(lambda x:  keras.backend.spatial_2d_padding(x, padding=((2, 0), (2, 0))))(conv1) 
    
    conv2 = IR_model.get_layer(index=9).output 
    conv2 = keras.layers.Lambda(lambda x:  keras.backend.spatial_2d_padding(x, padding=((3, 0), (3, 0))))(conv2) 
    
    conv3 = IR_model.get_layer(index=16).output 
    conv3 = keras.layers.Lambda(lambda x:  keras.backend.spatial_2d_padding(x, padding=((4, 0), (4, 0))))(conv3) 
    
    conv3a = IR_model.get_layer(index=266).output
    conv3a = keras.layers.Lambda(lambda x:  keras.backend.spatial_2d_padding(x, padding=((3, 0), (3, 0))))(conv3a) 

    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv3a)
    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv4)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool4)
    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv5)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5)

    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool5)
    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv6)
    pool6 = MaxPooling2D((2, 2), strides=(2, 2))(conv6)

    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool6)
    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv7)
    
    up8 = concatenate([Conv2DTranspose(384, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv7), conv6], axis=3)
    conv8 = Conv2D(384, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up8)

    up9 = concatenate([Conv2DTranspose(256, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv8), conv5], axis=3)
    conv9 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up9)

    up10 = concatenate([Conv2DTranspose(192, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv9), conv4], axis=3)
    conv10 = Conv2D(192, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up10)

    up11 = concatenate([Conv2DTranspose(128, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv10), conv3], axis=3)
    conv11 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up11)

    up12 = concatenate([Conv2DTranspose(64, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv11), conv2], axis=3)
    conv12 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up12)

    up13 = concatenate([Conv2DTranspose(32, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv12), conv1], axis=3)
    conv13 = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up13)

    conv13 = Conv2D(N_CHANNEL, (1, 1))(conv13)
    conv13 = Activation("sigmoid")(conv13)
    model = Model(img_input, conv13)
    Adam = optimizers.Adam(lr=LR)
    model.compile(optimizer=Adam, loss=dice_coef_loss, metrics=['accuracy', dice_coef, mean_iou])
    model.name = 'unet+IR'
    return model

In [ ]:
# UNET with initialization from DENSENT
def get_unet_Densenet201(LR=1e-4):
    from keras.applications.densenet import DenseNet201
    img_input = Input( (IMG_SIZE, IMG_SIZE, N_CHANNEL))

    Densenet_model = DenseNet201(weights='imagenet', input_tensor=img_input, include_top=False)
    
    conv1 = Densenet_model.get_layer(index=1).output 
    conv1 = Conv2D(64, (7, 7), activation="relu", kernel_initializer="he_normal")(conv1)    
    
    conv2 = Densenet_model.get_layer(index=4).output 
    conv3 = Densenet_model.get_layer(index=51).output 
    conv3a = Densenet_model.get_layer(index=139).output

    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv3a)
    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv4)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool4)
    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv5)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5)

    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool5)
    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv6)
    pool6 = MaxPooling2D((2, 2), strides=(2, 2))(conv6)

    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool6)
    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv7)
    
    up8 = concatenate([Conv2DTranspose(384, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv7), conv6], axis=3)
    conv8 = Conv2D(384, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up8)

    up9 = concatenate([Conv2DTranspose(256, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2),  padding='same')(conv8), conv5], axis=3)
    conv9 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal",  padding='same')(up9)

    up10 = concatenate([Conv2DTranspose(192, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2),  padding='same')(conv9), conv4], axis=3)
    conv10 = Conv2D(192, (3, 3), activation="relu", kernel_initializer="he_normal",  padding='same')(up10)

    up11 = concatenate([Conv2DTranspose(128, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2),  padding='same')(conv10), conv3], axis=3)
    conv11 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal",  padding='same')(up11)

    up12 = concatenate([Conv2DTranspose(64, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2),  padding='same')(conv11), conv2], axis=3)
    conv12 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal",  padding='same')(up12)

    up13 = concatenate([Conv2DTranspose(32, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2),  padding='same')(conv12), conv1], axis=3)
    conv13 = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal",  padding='same')(up13)

    conv13 = Conv2D(N_CHANNEL, (1, 1))(conv13)
    conv13 = Activation("sigmoid")(conv13)
    model = Model(img_input, conv13)
    
    Adam = optimizers.Adam(lr=LR)
    model.compile(optimizer=Adam, loss=dice_coef_loss, metrics=['accuracy', dice_coef, mean_iou])
    model.name = 'unet+dense'
    return model

In [ ]:
# UNET with initialization from RESNET
def get_unet_res(LR=1e-4):
    from keras.applications.resnet50 import ResNet50
    img_input = Input( (IMG_SIZE, IMG_SIZE, N_CHANNEL))
    resnet_model = ResNet50(weights='imagenet', input_tensor=img_input, include_top=False)

    conv1 = resnet_model.get_layer(index=1).output 
    conv1 = Conv2D(64, (7, 7), activation="relu", kernel_initializer="he_normal")(conv1)
    conv2 = resnet_model.get_layer(index=4).output 
    conv3 = resnet_model.get_layer(index=38).output 
    conv3a = resnet_model.get_layer(index=80).output 

    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv3a)
    conv4 = Conv2D(384, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv4)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool4)
    conv5 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv5)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5)

    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool5)
    conv6 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv6)
    pool6 = MaxPooling2D((2, 2), strides=(2, 2))(conv6)

    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(pool6)
    conv7 = Conv2D(512, (3, 3), activation="relu", padding='same', kernel_initializer="he_normal")(conv7)

    up8 = concatenate([Conv2DTranspose(384, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv7), conv6], axis=3)
    conv8 = Conv2D(384, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up8)

    up9 = concatenate([Conv2DTranspose(256, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv8), conv5], axis=3)
    conv9 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal",  padding='same')(up9)

    up10 = concatenate([Conv2DTranspose(192, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv9), conv4], axis=3)
    conv10 = Conv2D(192, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up10)

    up11 = concatenate([Conv2DTranspose(128, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv10), conv3], axis=3)
    conv11 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up11)

    up12 = concatenate([Conv2DTranspose(64, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv11), conv2], axis=3)
    conv12 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up12)

    up13 = concatenate([Conv2DTranspose(32, (3, 3), activation="relu", kernel_initializer="he_normal", strides=(2, 2), padding='same')(conv12), conv1], axis=3)
    conv13 = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding='same')(up13)

    conv13 = Conv2D(N_CHANNEL, (1, 1))(conv13)
    conv13 = Activation("sigmoid")(conv13)
    model = Model(img_input, conv13)
    Adam = optimizers.Adam(lr=LR)
    model.compile(optimizer=Adam, loss=dice_coef_loss, metrics=['accuracy', dice_coef, mean_iou])
    model.name = 'unet+res'
    return model

# Train

In [ ]:
# helper function to train and record scores in df_score
def train_model(model, df_score, training_generator, validation_generator):
    print(model.name)
    history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=EPOCH, use_multiprocessing=True, workers=4, verbose=1)
    df_score = update_df_score(df_score, history, model.name)
    del model
    return df_score

In [ ]:
# build training, validation generators and create df_score
df_train, df_val = get_train_val_df(frac=FRAC, train_val_split=TRAIN_VAL_SPLIT, random_state=SAMPLE_N, file_path=os.path.join(DATA_INPUT_DIR, 'stage_2_train_labels.csv'))
print('train size:', df_train.shape, '\tval size:', df_val.shape)
training_generator = datagen(df_train, batch_size=BATCH_SIZE, augment=True, predict=False)
validation_generator = datagen(df_val, batch_size=BATCH_SIZE, augment=True, predict=False)
df_score = pd.DataFrame(columns=['epoch', 'model', 
                                 'train_loss', 'val_loss', 
                                 'train_accuracy', 'val_accuracy',
                                 'train_dice_coef', 'val_dice_coef', 
                                 'train_mean_iou', 'val_mean_iou'])

In [ ]:
for model in [get_unet_vgg(), get_unet_res(), get_unet_IR(), get_unet_Densenet201()]:
    df_score = train_model(model, df_score, training_generator, validation_generator)
df_score

In [ ]:
file_name = '/kaggle/working/df_score_model_{}.csv'.format(SAMPLE_N)
df_score.to_csv(file_name) 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(28,7))
title = ['Train Loss', 'Train Accuracy', 'Train mean_iou', 'Train dice_coef']
model = ['unet+vgg', 'unet+res', 'unet+IR', 'unet+dense']
plot = ['train_loss', 'train_accuracy', 'train_mean_iou', 'train_dice_coef']

for i in range(4):
    df_score.loc[df_score['model'] == model[i]].plot(x='epoch', y=plot[i], label=model[i], ax=ax[i])
    _ = ax[i].grid()
    _ = ax[i].set_title(title[i])   
    _ = ax[i].set_xticks(np.arange(EPOCH))